In [ ]:
import sys
import matplotlib
matplotlib.use("Qt5Agg")
import numpy as np
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.gridspec as gridspec
#!pip install pyqt5
from PyQt5.QtWidgets import QApplication,QLineEdit,QGridLayout,QComboBox, QSizePolicy,QGroupBox, QWidget, QMainWindow, QMenu, QVBoxLayout, QSpinBox, QPushButton
from Plater import Plater , deviceChannels, RunIV , Plate , PlatingLoop
from PlaterData import ConductanceDatabaseObject,Grapher_Send 
from PyQt5.QtCore import Qt

#a procedure for setting the parameters for the plating.  You can set parameters for each channel or just one for whole chip.
def setParameters(channel):
    if channel.startswith('N'):
        bothElectrodes=True
        threshold_nA=10
        bias_V=1
        shortedThreshold_nS=8
        maxTimeS=8
        settleTime_S=1
        quickPulse =True
    elif channel.startswith('E'):
        bothElectrodes=True
        threshold_nA=10
        bias_V=1.2
        shortedThreshold_nS=8
        maxTimeS=8
        settleTime_S=1
        quickPulse =True
    elif channel.startswith('S'):
        bothElectrodes=False
        threshold_nA=10
        bias_V=1
        shortedThreshold_nS=8
        maxTimeS=120
        settleTime_S=1
        quickPulse =True
    elif channel.startswith('W'):
        bothElectrodes=True
        threshold_nA=10
        bias_V=1 
        shortedThreshold_nS=8
        maxTimeS=45
        settleTime_S=2
        quickPulse =True

    
    infos={ 
            #switch from electrode to electrode during plating (maxTimeS sets swithcing frequecy)
            "bothElectrodes":bothElectrodes,
            #current to stop applying a voltage
            "threshold_nA":threshold_nA,
            #if the current is higher than this, the electrode is shorted and do not plate
            "shortedThreshold_nS":shortedThreshold_nS,
            #maximum time to try to get a threshold event per electrode
            "maxTimeS":maxTimeS,
            #how long after the ramp to set the threshold to threshold_nA
            "settleTime_S":settleTime_S,
            #run a quick pulse before plating to condition the electrode
            "quickPulse":quickPulse,
             #maximum time to try to get a threshold event
            "maxTime_Min":10,
            #after plating IV check quit if the current is higher than this
            'finishThreshold_nA':4,
            #max number of times the program will run an IV, check against finishThreshold_nA and repeat the plating
            'platingAttempts':10,
            #max number of times the program will switch the electrode and do a CV against the threshold_nA over maxTimeS
            'attemptsBeforeIV':10,
            #if finishThreshold_nA is not reached, try again?
            'redoAfterIV':True,
            #plating bias
            "bias_V":bias_V,
            
            
            #not really tested yet, dont use pulsing
            "pulsed":False,
             #reporting for server
            "Conductance":0,
            "Capacitance":0,
            "Wafer":wafer,
            "Chip":chip,
            "Device":channel,
            "Stage":stage,
            "source":"Plater",
            "dataFolder":f'//BIOD1633/Data/Plater/{wafer}/{chip}',
            "timestamp":str( datetime.now()),
            "bias":100,
            }    
    return infos 

class MyMplCanvas(FigureCanvas):
    def __init__(self, parent=None, width=10, height=7, dpi=200):
        
        self.fig = Figure(tight_layout=True,figsize=(width, height), dpi=dpi)
        gs = gridspec.GridSpec(2, 2)

        self.axes={}
        self.axes['IV'] = self.fig.add_subplot(gs[1,0])
        self.axes['IV'].plot(np.arange(0, 1e6, 1000))
        self.axes['IV'].set_ylabel('Current (nA)')
        self.axes['IV'].set_xlabel('Voltage (mV)')
        
        self.axes['Plate'] = self.fig.add_subplot(gs[0, :])
        self.axes['Plate'].plot(np.arange(1., 0., -0.1) * 2000., np.arange(1., 0., -0.1))
        self.axes['Plate'].set_ylabel('Current (nA)')
        self.axes['Plate'].set_xlabel('Time(s)')

        self.axes['Pulse'] =self. fig.add_subplot(gs[1, 1])
        self.axes['Pulse'].plot(np.arange(1., 0., -0.1) * 2000., np.arange(1., 0., -0.1))
        self.axes['Pulse'].set_ylabel('Current (nA)')
        self.axes['Pulse'].set_xlabel('Time(s)')
        
        
        #self.fig.align_labels()  # same as fig.align_xlabels(); fig.align_ylabels()
        
        
        FigureCanvas.__init__(self, self.fig)
        self.setParent(parent)
        self.axes['IV'] .plot()
        FigureCanvas.setSizePolicy(self,
                QSizePolicy.Expanding,
                QSizePolicy.Expanding)
        FigureCanvas.updateGeometry(self)

class PlatingGraphs(MyMplCanvas):
    """Simple canvas with a sine plot."""
    def update_IV(self, title,bias,current):
        self.axes['IV'].cla()
        self.axes['IV'].set_title(title)
        self.axes['IV'].plot(bias, current)
        self.draw()
        
    def update_Plate(self, current):
        self.axes['Plate'].cla()
        self.axes['Plate'].plot(current)
        self.draw()
        
    def update_Pulse(self, current):
        self.axes['Pulse'].cla()
        self.axes['Pulse'].plot(current)
        self.draw()        

class ApplicationWindow(QMainWindow):
    def Plate(self):
        return 0
    
    def IV(self):
        return 0
    
    def channelSelectionchange(self,channel):
        print(deviceChannels[channel])
    
    def waferChanged(self,wafer):
        self.wafer = wafer
    def chipChanged(self,chip):
        self.chip=chip
    def stageChanged(self,stage):
        self.stage =stage
    
    def __init__(self):
        super().__init__()
        
        self.wafer = "wafer"
        self.chip= "chip"
        self.stage = "stage"
        
        self.setStyleSheet('font-size: 15px;')
        self.file_menu = QMenu('&Actions', self)
        self.file_menu.addAction('&Plate', self.Plate, Qt.CTRL + Qt.Key_P)
        self.file_menu.addAction('&IV', self.IV, Qt.CTRL + Qt.Key_I)
        self.file_menu.addAction('&Quit', self.close, Qt.CTRL + Qt.Key_Q)
        self.menuBar().addMenu(self.file_menu)

        self.main_widget = QWidget()
        layout = QVBoxLayout(self.main_widget)

        self.horizontalGroupBox = QGroupBox("Grid")
        glayout = QGridLayout()
        #glayout.setColumnStretch(1, 4)
        #glayout.setColumnStretch(2, 4)
        
        self.spinbox = QComboBox( )
        for device in deviceChannels:
            self.spinbox.addItem(device)

        self.sc = PlatingGraphs(self.main_widget)
        
        
        self.waferEdit = QLineEdit()
        self.waferEdit.textChanged.connect(self.waferChanged)
        
        self.chipEdit = QLineEdit()
        self.chipEdit.textChanged.connect(self.chipChanged)
        
        self.stageEdit = QLineEdit()
        self.stageEdit.textChanged.connect(self.stageChanged)
        
        self.plateButton=QPushButton('Plate')
        self.IVButton=QPushButton('Take IVs')
        
        glayout.addWidget(self.waferEdit,0,0)
        glayout.addWidget(self.chipEdit,0,1)
        glayout.addWidget(self.stageEdit,0,2)
        glayout.addWidget(self.spinbox,0,3)
        
        glayout.addWidget(self.plateButton,0,4)
        glayout.addWidget(self.IVButton,0,5)
        
        
        glayout.addWidget(self.sc,1,0,1,6)
       
        self.spinbox.currentIndexChanged.connect(self.channelSelectionchange)
        self.plateButton.clicked.connect(self.Plate)
        self.IVButton.clicked.connect(self.IV)
        
        

        #self.spinbox.valueChanged.connect(self.sc.update_figure)

        #self.sc.update_figure(self.spinbox.value())

        
        self.horizontalGroupBox.setLayout(glayout)  
      
        layout.addWidget(self.horizontalGroupBox)

        self.main_widget.setFocus()
        self.setCentralWidget(self.main_widget)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    win = ApplicationWindow()
    win.setWindowTitle("PyQt5 Matplotlib App Demo")
    win.show()
    app.exec_()

In [ ]:
plater=Plater(10000) 

In [ ]:
wafer='T011'
chip ='A4'

platingChannels = [x for x in deviceChannels]
platingChannels 




conductanceDatabase=ConductanceDatabaseObject(wafer,chip) 
 
repeatChannels=[]


In [ ]:
#########################################  Plating loop ##################################
stage ='Plating'

PlatingLoop(stage, plater, setParameters,platingChannels,repeatChannels,conductanceDatabase,dataSend)
print('Overtimed Channels')        
print(repeatChannels)        

In [ ]:
#########################################  Perform just the IV ##################################
numberScans=1

stage ='Empty' 
rtBias_mV = 300

topElectrode =True

for i in range(numberScans):
    for channel in plater.channels:
        print(channel)
        plater.setBias(0)
        plater.SelectChannel(channel)
        time.sleep(.5)
        infos = setParameters(channel)
        RunIV(infos, channel, stage+ '_'+ str(i),  rtBias_mV, plater, conductanceDatabase, slew_mV_s=3000, topElectrode=topElectrode)
 
    conductanceDatabase.PlotHistory()
    
            
    if numberScans>1:
        for j in range(60*3):
            time.sleep(20)

In [ ]:
import pyvisa
import time
import numpy as np
import matplotlib.pyplot as plt
rm = pyvisa.ResourceManager()
print(rm.list_resources())

keithley = rm.open_resource("TCPIP0::129.219.2.85::5025::SOCKET")
print('\n Open Successful!')
keithley.read_termination = '\n'
keithley.write_termination = '\n'

code ="""
reset()
loadscript
    
    
    numCycles=2
    frequency=45
    limitI=10e-6

	local COMPLETE = "{COMPLETE}"
	-- Generate the source values
	local Vpp				= .2
	local sourceValues		= {} 
	local pointsPerCycle	= 7200 / frequency
	local numDataPoints		= pointsPerCycle * numCycles

	local res = {}
    for i = 1, numDataPoints do
        res[i] =math.abs( math.sin(i * 2 * math.pi / pointsPerCycle))
        sourceValues[i] = (Vpp * res[i])  
    end

	-- Configure the SMU ranges
	smua.reset()
	smua.source.settling		= smua.SETTLE_FAST_POLARITY
	smua.source.autorangev		= smua.AUTORANGE_OFF
	smua.source.autorangei		= smua.AUTORANGE_OFF
    smua.source.rangev			= math.abs( Vpp ) +.01
	smua.source.delay			= 0
	smua.source.limiti			= 1e-3

	smua.measure.autorangev		= smua.AUTORANGE_OFF
	smua.measure.autorangei		= smua.AUTORANGE_OFF
	smua.measure.autozero		= smua.AUTOZERO_OFF
	smua.measure.delay			= 0
	smua.measure.delayfactor    = 1
	smua.measure.analogfilter   = 0

	-- Voltage will be measured on the same range as the source range
	smua.measure.rangei			= limitI
	smua.measure.nplc			= 0.001

	-- Prepare the Reading Buffers
	smua.nvbuffer1.clear()
	smua.nvbuffer1.collecttimestamps	= 1
	smua.nvbuffer2.clear()
	smua.nvbuffer2.collecttimestamps	= 1

	-- Configure the trigger model
	--============================
	
	-- Timer 1 controls the time between source points
	trigger.timer[1].delay = (1 / 7200)
	trigger.timer[1].passthrough = true
	trigger.timer[1].stimulus = smua.trigger.ARMED_EVENT_ID
	trigger.timer[1].count = numDataPoints - 1

	-- Configure the SMU trigger model
	smua.trigger.source.listv(sourceValues)
	smua.trigger.source.limiti		= limitI
	smua.trigger.measure.action		= smua.ENABLE
	smua.trigger.measure.iv(smua.nvbuffer1, smua.nvbuffer2)
	smua.trigger.endpulse.action	= smua.SOURCE_HOLD
	smua.trigger.endsweep.action	= smua.SOURCE_IDLE
	smua.trigger.count				= numDataPoints
	smua.trigger.arm.stimulus		= 0
	smua.trigger.source.stimulus	= trigger.timer[1].EVENT_ID
	smua.trigger.measure.stimulus	= 0
	smua.trigger.endpulse.stimulus	= 0
	smua.trigger.source.action		= smua.ENABLE
	-- Ready to begin the test

	smua.source.output					= smua.OUTPUT_ON
	-- Start the trigger model execution
	smua.trigger.initiate()
	-- Wait until the sweep has completed
	waitcomplete()
	smua.source.output					= smua.OUTPUT_OFF

 
	printbuffer(1, smua.nvbuffer1.n, smua.nvbuffer1.readings)
	print('')
    printbuffer(1, smua.nvbuffer2.n, smua.nvbuffer2.readings)

	print('Done')
endscript
script.run()
""".split('\n')

keithley.query("print('h')")
def KeithleyIV():
    for line in code:
        keithley.write(line)

    time.sleep(2)
    text=keithley.read()
    while 'Done' not in text :
        text+=keithley.read() + "\n"

    text = text.split('\n') 
    current = np.array( [float(x) for x in text[0].split(',')[:-1]])
    volts  = np.array( [float(x) for x in text[1].split(',')[:-1]])
    plt.plot(volts,current)
    plt.show()
    slope=np.polyfit(volts,current,1)[0]
    
    return slope,volts,current*1e9

In [ ]:
#########################################  Perform just the IV ##################################
stage ='DNA_2K_L'
rtBias_mV = 50

if os.path.exists(f'C:/Data/Plater/{wafer}')==False:
    os.mkdir(f'C:/Data/Plater/{wafer}')
if os.path.exists(f'C:/Data/Plater/{wafer}/{chip}')==False:    
    os.mkdir(f'C:/Data/Plater/{wafer}/{chip}')

channel='N8'    
filename=f'C:/Data/Plater/{wafer}/{chip}/{channel}_{stage}'
if os.path.exists(filename + '.npy'):
    raise Exception('Wrong Path')    
    pass

with Plater(10000) as plater:
        plater.setBias(0)
        plater.TopElectrode()
        
        plater.setThreshold(current_nA = 200)
        plater.disableThreshold()

        for channel in plater.channels:
            print(channel)
            plater.setBias(0)
            plater.SelectChannel(channel)
            time.sleep(1)
            
            filename=f'C:/Data/Plater/{wafer}/{chip}/{channel}_{stage}'
            slope, outBias,currents =KeithleyIV()
            plater.setBias(0)
            
            
            with open(filename+ '_KIV.npy', 'wb') as f:
                np.save(f, slope)
                np.save(f, outBias)
                np.save(f, currents)

In [ ]:

with Plater(10000) as plater:
        plater.setBias(0)
        plater.TopElectrode()
        
        plater.setThreshold(current_nA = 200)
        plater.disableThreshold()

        for channel in plater.channels:
            plater.setBias(0)
            plater.SelectChannel(channel)
            time.sleep(1)
            plater.TopElectrode()
            times,currents=plater.runRT(voltage_mV=100,time_s= 3)
            plt.plot(currents)
            
            plater.BottomElectrode()
            times,currents=plater.runRT(voltage_mV=100,time_s= 3)
            plt.plot(currents)
            
            plt.show()
            
            plater.TopElectrode()
            slope, outBias,currents =plater.runIV(maxVoltage=.25,slew_mV_s=1000, samplesPerPoint=200)
             
            
            plater.BottomElectrode()
            slope, outBias,currents =plater.runIV(maxVoltage=.25,slew_mV_s=1000, samplesPerPoint=200)
             
            
            
            
            break
            

In [ ]:
plater = Plater(10000)
plater.setBias(0)
plater.TopElectrode()
#plater.BottomElectrode()

plater.setThreshold(current_nA = 200)
plater.disableThreshold()

            
plater.setBias(1)
plater.SelectChannel('N1')

In [ ]:
plater.setBias(1)
plater.ResetDevice()
plater.SelectChannel('E8')

In [ ]:
N1->W8
N8->W1
W8->N1
W1->N8


